In [2]:
execfile('../src/utils.py')
execfile('../src/cellslib.py')

In [3]:
path = '/notebooks/moldev-data/original/06-19-2015/6.19.15 All ssC data - Jonah Simon.txt'
savepath = '/notebooks/moldev-data/analyzed/06-19-2015/ssC.csv'

conditions = [('PML ssC w RNase A',24),
              ('PML APB w RNase A',25),
              ('PML ssC wo RNase A',26),
              ('PML IF only control no EtOH dehydrat',27),
              ('PML ssC control no FISH',28),
              ('IgG APB w LNA probe before denature',29),
              ('IgG APB w LNA probe after denature',30),
              ('IgG ssC w LNA probe no denature',31)]

cell_dict = {"U2OS": ['A01','A02','B01','B02'],
             "143B": ['A03','A04','B03','B04']}

date = '06-18-2015'

In [4]:
well_df = create_well_df(cell_dict)
condition_lookup = pd.concat([create_plate_df(well_df,a,b) for a,b in conditions])
condition_lookup['Date'] = date

In [5]:
condition_lookup.head()

,Well Name,Plate Name,Condition,Plate ID,Date
0,A03,24,PML ssC w RNase A 143B,b0d5debe5142,06-18-2015
1,A04,24,PML ssC w RNase A 143B,b0d5debe5142,06-18-2015
2,B03,24,PML ssC w RNase A 143B,b0d5debe5142,06-18-2015
3,B04,24,PML ssC w RNase A 143B,b0d5debe5142,06-18-2015
0,A01,24,PML ssC w RNase A U2OS,b0d5debe5142,06-18-2015


In [6]:
# String -> String
def rename_column(col):
    """ Rename column col to remove whitespace, backslashes, prefixes,
        and suffixes (esp. large parenthetic suffix). """
    if col.startswith('Cell:'):
        return col.split('(')[0].lstrip("Cell:").rstrip('/').strip(' ')
    else:
        return col.split('(')[0].rstrip('/').strip(' ')

cell_config = dict(
    path = path,
    plate_delimiter = "ATF",
    delimiter = '\t',
    skiprows = 4,
    dropcols = ['Cell ID',
                'Site ID',
                'MEASUREMENT SET ID',
                '.*ObjectID.*',
                '\.[0-9]*\Z'],
    normcols = [['Normalized APB spots',
                  ['# of APBs'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized Coloc area',
                  ['Area_Coloc_Avg'],
                  ['Area_FITC','Area_TxRed']],
                ['Normalized Coloc spots',
                  ['# Coloc Spots'],
                  ['# of FITC spots', '# of TxRed spots']],
                ['Normalized Coloc spots (by FITC)',
                  ['# Coloc Spots'],
                  ['# of FITC spots']],
                ['Normalized Coloc spots (by TxRed)',
                  ['# Coloc Spots'],
                  ['# of TxRed spots']]],
    colrename = rename_column,
    check = identity
    )

cell_data = get_cell_data(cell_config)

In [7]:
data = pd.merge(cell_data.rename(columns={"Plate ID":"Plate Name"}),
                condition_lookup,
                on = ['Plate Name','Well Name'])

# Write to file
data.to_csv(savepath,index=False)

In [8]:
data.head()

,Well Name,Plate Name,# of FITC spots,Area_FITC,NucArea_Avg,NucIntegrated Intensity_Average,Integrated Intensity_FITC,Area_TxRed,Integrated Intensity_TxRed,# of TxRed spots,...,FITC-TxRed Spots,Laser focus score,Normalized APB spots,Normalized Coloc area,Normalized Coloc spots,Normalized Coloc spots (by FITC),Normalized Coloc spots (by TxRed),Condition,Plate ID,Date
0,A01,28,9,0.314608,166.787186,6676052,15065.444336,0.101124,1533,1,...,0,29,0,NaN,0,0,0,PML ssC control no FISH U2OS,10cfebc04e76,06-18-2015
1,A01,28,30,0.380900,220.989655,7825328,45452.234375,NaN,NaN,0,...,0,29,0,NaN,0,0,NaN,PML ssC control no FISH U2OS,10cfebc04e76,06-18-2015
2,A01,28,13,0.294729,154.607361,6503341,13281.845703,NaN,NaN,0,...,0,29,0,NaN,0,0,NaN,PML ssC control no FISH U2OS,10cfebc04e76,06-18-2015
3,A01,28,13,0.279171,219.866043,11076027,21914.615234,NaN,NaN,0,...,0,29,0,NaN,0,0,NaN,PML ssC control no FISH U2OS,10cfebc04e76,06-18-2015
4,A01,28,9,0.463173,88.404846,4023999,35109.667969,NaN,NaN,0,...,0,31,0,NaN,0,0,NaN,PML ssC control no FISH U2OS,10cfebc04e76,06-18-2015
